In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/20-newsgroups/misc.forsale.txt
/kaggle/input/20-newsgroups/rec.autos.txt
/kaggle/input/20-newsgroups/list.csv
/kaggle/input/20-newsgroups/comp.os.ms-windows.misc.txt
/kaggle/input/20-newsgroups/sci.electronics.txt
/kaggle/input/20-newsgroups/comp.sys.mac.hardware.txt
/kaggle/input/20-newsgroups/talk.politics.mideast.txt
/kaggle/input/20-newsgroups/talk.politics.guns.txt
/kaggle/input/20-newsgroups/talk.religion.misc.txt
/kaggle/input/20-newsgroups/comp.graphics.txt
/kaggle/input/20-newsgroups/soc.religion.christian.txt
/kaggle/input/20-newsgroups/rec.sport.hockey.txt
/kaggle/input/20-newsgroups/rec.sport.baseball.txt
/kaggle/input/20-newsgroups/comp.windows.x.txt
/kaggle/input/20-newsgroups/comp.sys.ibm.pc.hardware.txt
/kaggle/input/20-newsgroups/rec.motorcycles.txt
/kaggle/input/20-newsgroups/sci.med.txt
/kaggle/input/20-newsgroups/sci.space.txt
/kaggle/input/20-newsgroups/alt.atheism.txt
/kaggle/input/20-newsgroups/sci.crypt.txt
/kaggle/input/20-newsgroups/talk.politics

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import re
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split  # Add this import

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define readable category names
readable_category_names = {
    'alt.atheism': 'Atheism',
    'comp.graphics': 'Graphics',
    'comp.os.ms-windows.misc': 'Windows',
    'comp.sys.ibm.pc.hardware': 'PC Hardware',
    'comp.sys.mac.hardware': 'Mac Hardware',
    'comp.windows.x': 'X Windows',
    'misc.forsale': 'For Sale',
    'rec.autos': 'Autos',
    'rec.motorcycles': 'Motorcycles',
    'rec.sport.baseball': 'Baseball',
    'rec.sport.hockey': 'Hockey',
    'sci.crypt': 'Cryptography',
    'sci.electronics': 'Electronics',
    'sci.med': 'Medicine',
    'sci.space': 'Space',
    'soc.religion.christian': 'Christianity',
    'talk.politics.guns': 'Guns',
    'talk.politics.mideast': 'Middle East Politics',
    'talk.politics.misc': 'General Politics',
    'talk.religion.misc': 'Religion'
}

# Load the dataset
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))

# Extract features (text) and labels (categories)
X, y = newsgroups.data, newsgroups.target

# Map target names to readable names
readable_target_names = [readable_category_names[name] for name in newsgroups.target_names]

# Split the dataset into 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Print dataset information
print("Training set size:", len(X_train))
print("Test set size:", len(X_test))
print("Categories:", readable_target_names)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Training set size: 9051
Test set size: 2263
Categories: ['Atheism', 'Graphics', 'Windows', 'PC Hardware', 'Mac Hardware', 'X Windows', 'For Sale', 'Autos', 'Motorcycles', 'Baseball', 'Hockey', 'Cryptography', 'Electronics', 'Medicine', 'Space', 'Christianity', 'Guns', 'Middle East Politics', 'General Politics', 'Religion']


In [3]:
# Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join back into a string
    return " ".join(tokens)

# Apply preprocessing to the dataset
X_train_clean = [preprocess_text(doc) for doc in X_train]
X_test_clean = [preprocess_text(doc) for doc in X_test]

In [4]:
# Count the number of news articles in each category
train_category_counts = Counter([readable_target_names[label] for label in y_train])
test_category_counts = Counter([readable_target_names[label] for label in y_test])

print("\nTraining Set Category Distribution:")
for category, count in train_category_counts.items():
    print(f"{category}: {count}")

print("\nTest Set Category Distribution:")
for category, count in test_category_counts.items():
    print(f"{category}: {count}")


Training Set Category Distribution:
Medicine: 475
Autos: 475
Motorcycles: 478
Electronics: 473
Middle East Politics: 451
PC Hardware: 472
Windows: 473
General Politics: 372
Mac Hardware: 463
Baseball: 478
Cryptography: 476
Space: 474
Graphics: 467
Atheism: 384
Christianity: 479
Hockey: 480
X Windows: 474
Religion: 302
Guns: 437
For Sale: 468

Test Set Category Distribution:
X Windows: 119
Mac Hardware: 115
Graphics: 117
PC Hardware: 118
Cryptography: 119
Autos: 119
Middle East Politics: 113
For Sale: 117
Christianity: 120
Space: 119
Religion: 75
Guns: 109
Windows: 118
Motorcycles: 120
Electronics: 118
Hockey: 120
Medicine: 119
Baseball: 119
Atheism: 96
General Politics: 93


In [6]:
# Import necessary libraries
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import KeyedVectors  # For loading pretrained Word2Vec
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK resources
nltk.download('punkt')

# Load pretrained Word2Vec model
print("Loading pretrained Word2Vec model...")
word2vec_model = KeyedVectors.load_word2vec_format(
    '/kaggle/input/word2vec-pretrained/GoogleNews-vectors-negative300.bin', binary=True
)
print("Pretrained Word2Vec model loaded.")

# Function to generate document embeddings using Word2Vec
def get_document_embedding(text, model, vector_size=300):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Get word vectors for tokens in the model's vocabulary
    embeddings = [model[word] for word in tokens if word in model]
    # Return the mean of the word vectors (or zeros if no valid words are found)
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(vector_size)

# Generate document embeddings for the training and test sets
print("Generating document embeddings for the training set...")
X_train_w2v = np.array([get_document_embedding(doc, word2vec_model) for doc in X_train_clean])

print("Generating document embeddings for the test set...")
X_test_w2v = np.array([get_document_embedding(doc, word2vec_model) for doc in X_test_clean])

# Check the shape of the resulting matrices
print("\nTraining Word2Vec Embeddings Shape:", X_train_w2v.shape)
print("Test Word2Vec Embeddings Shape:", X_test_w2v.shape)

# Train a Logistic Regression model on Word2Vec embeddings
print("\nTraining Logistic Regression on Word2Vec embeddings...")
clf_w2v = LogisticRegression(max_iter=1000, random_state=42)
clf_w2v.fit(X_train_w2v, y_train)

# Predict on the test set
y_pred_w2v = clf_w2v.predict(X_test_w2v)

# Evaluate performance
print("\nAccuracy with Word2Vec:", accuracy_score(y_test, y_pred_w2v))
print("\nClassification Report:\n", classification_report(y_test, y_pred_w2v, target_names=readable_target_names))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading pretrained Word2Vec model...
Pretrained Word2Vec model loaded.
Generating document embeddings for the training set...
Generating document embeddings for the test set...

Training Word2Vec Embeddings Shape: (9051, 300)
Test Word2Vec Embeddings Shape: (2263, 300)

Training Logistic Regression on Word2Vec embeddings...

Accuracy with Word2Vec: 0.6429518338488732

Classification Report:
                       precision    recall  f1-score   support

             Atheism       0.53      0.39      0.45        96
            Graphics       0.57      0.56      0.56       117
             Windows       0.58      0.50      0.54       118
         PC Hardware       0.52      0.58      0.54       118
        Mac Hardware       0.56      0.46      0.50       115
           X Windows       0.61      0.61      0.61       119
            For Sale       0.61      0.63      0.62   

In [7]:
# Identify misclassifications
misclassifications = []
for i in range(len(X_test)):
    if y_pred_w2v[i] != y_test[i]:  # Check if prediction is incorrect
        misclassifications.append({
            "Input": X_test[i],  # Original input text
            "Predicted": readable_target_names[y_pred_w2v[i]],  # Predicted category
            "Correct": readable_target_names[y_test[i]]  # Correct category
        })

# Print misclassifications
print("\nMisclassifications:")
for idx, misclassification in enumerate(misclassifications[:5]):
    print(f"Example {idx + 1}:")
    print(f"  Input: {misclassification['Input']}")
    print(f"  Predicted: {misclassification['Predicted']}")
    print(f"  Correct: {misclassification['Correct']}")
    print("-" * 80)

# Total number of misclassifications
print(f"Total Misclassifications: {len(misclassifications)}")


Misclassifications:
Example 1:
  Input: After reading reports from Germany of success in accelerating a Quadra
or Centris simply by changing the clock oscillator, I decided to test the
claim. I pulled out my Variable Speed Overdrive and the motherboard's
50 mhz clock chip. I put a socket in the clock's place and inserted a
64 mhz TTL clock oscillator I had left over from working on some SI's.
I can't believe it. It actually works. I'm not getting SCSI timing errors
either. This is only after a short run time but I'll keep posting results.
Did I spend all that money on the VSO for nothing? If this keeps working,
the lack of a double boot in itself will be worth the effort.
  Predicted: PC Hardware
  Correct: Mac Hardware
--------------------------------------------------------------------------------
Example 2:
  Input: : |> 
: |>         Is there any way to connect two pointing devices to one serial
: |>         port? I haven't tried this but I believe they would interfere
: |>       

### **Analysis of Misclassifications**

#### **1. Common Patterns in Misclassifications**
From the examples provided, we can observe a few recurring themes:

1. **Overlapping Themes Between Categories**:
   - Example 1: "Mac Hardware" vs. "PC Hardware"
     - The text discusses hardware components (e.g., clock oscillator), which are relevant to both Mac and PC hardware. However, the mention of "Quadra" or "Centris" (Mac-specific models) should have tipped the model toward "Mac Hardware."
   - Example 4: "PC Hardware" vs. "Autos"
     - The text discusses technical support and repairs for computers, but the model incorrectly predicts "Autos." This suggests that the embeddings failed to capture domain-specific terms like "techs," "warranty," or "computer."

2. **Ambiguous Language**:
   - Example 5: "Christianity" vs. "Medicine"
     - The text uses metaphorical language ("medicine for the soul") to describe Christianity. While the word "medicine" is prominent, the context clearly relates to religion. The model likely focused too much on the literal term "medicine" rather than the broader context.

3. **Rare or Domain-Specific Terms**:
   - Example 3: "X Windows" vs. "Windows"
     - The text mentions "X-Appeal" and "X Windows"-specific directories (e.g., `/pub/MSDOS/xappeal`). However, the model incorrectly predicts "Windows," likely because it doesn’t recognize "X Windows" as a distinct category.
   - Example 2: "Graphics" vs. "PC Hardware"
     - The text discusses serial ports and modems, which are more closely related to hardware than graphics. However, the model predicts "Graphics," possibly due to insufficient training on hardware-related terms.

4. **Lack of Contextual Understanding**:
   - Word2Vec embeddings capture semantic relationships between words but lack contextual understanding. For example:
     - In Example 5, the model fails to interpret the metaphorical use of "medicine."
     - In Example 3, the model doesn’t recognize "X Windows" as a specific operating system.

### **Why Are These Errors Happening?**

1. **Out-of-Vocabulary (OOV) Words**:
   - Pretrained Word2Vec embeddings may not include domain-specific terms like "Centris," "Quadra," or "X-Appeal." Documents containing such terms may result in zero vectors, leading to poor classification.

2. **Averaging Embeddings**:
   - The document embedding is computed as the average of word vectors, which can dilute important signals. For example, in Example 5, the word "medicine" dominates the embedding, overshadowing the religious context.

3. **Overlap Between Categories**:
   - Some categories (e.g., "PC Hardware" and "Mac Hardware") share similar vocabulary, making it harder for the model to distinguish between them.

4. **Lack of Contextual Information**:
   - Word2Vec embeddings don’t capture the order or context of words, which can lead to misinterpretations. For example, the metaphorical use of "medicine" in Example 5 is lost.

### **How Can We Improve the Model?**

#### **2. Fine-Tune Word2Vec Embeddings**
- Fine-tune the pretrained Word2Vec model on your dataset to include domain-specific terms (e.g., "X Windows," "Centris").
- This will help the model better represent rare or specialized vocabulary.

#### **3. Weighted Averaging**
- Instead of averaging all word vectors equally, weight them by their TF-IDF scores. This ensures that important words (e.g., "Centris" in Example 1) contribute more to the document embedding.

#### **4. Group Similar Categories**
- Combine categories with overlapping themes (e.g., "PC Hardware" and "Mac Hardware" → "Computers & Technology"). This reduces ambiguity and improves classification accuracy.
---

### **Key Takeaways**
1. **Strengths of Word2Vec**:
   - Captures semantic relationships between words (e.g., "car" and "vehicle").
   - Works well for tasks where word similarity matters.

2. **Limitations of Word2Vec**:
   - Struggles with domain-specific terms and OOV words.
   - Lacks contextual understanding, leading to errors in ambiguous cases.

Weighted Averaging
Instead of averaging all word vectors equally, weight them by their TF-IDF scores. This ensures that important words (e.g., "Centris" in Example 1) contribute more to the document embedding.

In [8]:
# Import necessary libraries
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import KeyedVectors  # For loading pretrained Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

# Download NLTK resources
nltk.download('punkt')

# Load pretrained Word2Vec model
print("Loading pretrained Word2Vec model...")
word2vec_model = KeyedVectors.load_word2vec_format(
    '/kaggle/input/word2vec-pretrained/GoogleNews-vectors-negative300.bin', binary=True
)
print("Pretrained Word2Vec model loaded.")

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data to get TF-IDF scores
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_clean)

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test_clean)

# Get the vocabulary from the TF-IDF vectorizer
vocabulary = tfidf_vectorizer.get_feature_names_out()

# Function to generate weighted document embeddings using Word2Vec + TF-IDF
def get_weighted_document_embedding(text, model, tfidf_vectorizer, vector_size=300):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Get the TF-IDF scores for the document
    tfidf_scores = dict(zip(vocabulary, tfidf_vectorizer.transform([text]).toarray()[0]))
    # Initialize variables for weighted sum and total weight
    weighted_sum = np.zeros(vector_size)
    total_weight = 0
    # Iterate over tokens and compute the weighted sum
    for word in tokens:
        if word in model and word in tfidf_scores:
            weight = tfidf_scores[word]
            weighted_sum += weight * model[word]
            total_weight += weight
    # Return the weighted average (or zeros if no valid words are found)
    return weighted_sum / total_weight if total_weight > 0 else np.zeros(vector_size)

# Generate weighted document embeddings for the training and test sets
print("Generating weighted document embeddings for the training set...")
X_train_w2v_tfidf = np.array([get_weighted_document_embedding(doc, word2vec_model, tfidf_vectorizer) for doc in X_train_clean])

print("Generating weighted document embeddings for the test set...")
X_test_w2v_tfidf = np.array([get_weighted_document_embedding(doc, word2vec_model, tfidf_vectorizer) for doc in X_test_clean])

# Check the shape of the resulting matrices
print("\nTraining Weighted Word2Vec Embeddings Shape:", X_train_w2v_tfidf.shape)
print("Test Weighted Word2Vec Embeddings Shape:", X_test_w2v_tfidf.shape)

# Train a Logistic Regression model on weighted Word2Vec embeddings
print("\nTraining Logistic Regression on Weighted Word2Vec embeddings...")
clf_w2v_tfidf = LogisticRegression(max_iter=1000, random_state=42)
clf_w2v_tfidf.fit(X_train_w2v_tfidf, y_train)

# Predict on the test set
y_pred_w2v_tfidf = clf_w2v_tfidf.predict(X_test_w2v_tfidf)

# Evaluate performance
print("\nAccuracy with Weighted Word2Vec:", accuracy_score(y_test, y_pred_w2v_tfidf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_w2v_tfidf, target_names=readable_target_names))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading pretrained Word2Vec model...
Pretrained Word2Vec model loaded.
Generating weighted document embeddings for the training set...
Generating weighted document embeddings for the test set...

Training Weighted Word2Vec Embeddings Shape: (9051, 300)
Test Weighted Word2Vec Embeddings Shape: (2263, 300)

Training Logistic Regression on Weighted Word2Vec embeddings...

Accuracy with Weighted Word2Vec: 0.589041095890411

Classification Report:
                       precision    recall  f1-score   support

             Atheism       0.39      0.29      0.34        96
            Graphics       0.56      0.54      0.55       117
             Windows       0.54      0.43      0.48       118
         PC Hardware       0.50      0.48      0.49       118
        Mac Hardware       0.50      0.44      0.47       115
           X Windows       0.57      0.62      0.60       119
 